## Inference

In [6]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from models import build_model

/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
WARNING[XFORMERS]: xFormers can't load C++/CUDA extensions. xFormers was built for:
    PyTorch 2.0.0+cu118 with CUDA 1108 (you have 2.5.1+cu124)
    Python  3.10.11 (you have 3.10.15)
  Please reinstall xformers (see https://github.com/facebookresearch/xformers#installing-xformers)
  Memory-efficient attention, SwiGLU, sparse and more won't be available.
  Set XFORMERS_MORE_DETAILS=1 for more details
/home/jovyan/.mlspace/envs/

Please 'pip install apex'
Please 'pip install apex'
Please 'pip install apex'
Please 'pip install apex'


/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/diffusers/utils/outputs.py:63: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(
/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/transformers/utils/generic.py:311: FutureWarning: `torch.utils._pytree._register_pytree_node` is deprecated. Please use `torch.utils._pytree.register_pytree_node` instead.
  torch.utils._pytree._register_pytree_node(


In [2]:
import os
import sys
import torch
import numpy as np
import random
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from models import build_model

model_path = "/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft"#"/home/jinyang06/models/VideoLaVIT-v1/language_model_sft"
model_dtype='bf16'

max_video_clips = 16
device_id = 0
torch.cuda.set_device(device_id)
device = torch.device('cuda')

seed = 42
#torch.manual_seed(seed)
np.random.seed(seed)
random.seed(seed)

# For Multi-Modal Understanding
runner = build_model(model_path=model_path, model_dtype=model_dtype, understanding=True, 
        device_id=device_id, use_xformers=False, max_video_clips=max_video_clips,)

Loading Video LaVIT Model Weight from /home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft, model precision: bf16
Not used {}


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

Some weights of the model checkpoint at /home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/language_model_sft were not used when initializing VideoLaVITLlamaForCausalLM: ['model.motion_tokenizer.quantize.embedding.cluster_size', 'model.motion_tokenizer.quantize.embedding.initted', 'model.motion_tokenizer.quantize.cluster_size', 'model.motion_tokenizer.quantize.embedding.embed_avg']
- This IS expected if you are initializing VideoLaVITLlamaForCausalLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing VideoLaVITLlamaForCausalLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


The Visual Vocab Size is 16384
The llama tokenizer vocab size is 32000
The maximal clip number is 16


In [3]:
video_path = '/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/eval/videoChatgpt_test/v_hFi6S_guB7I.mp4'
prompt = 'What is the appearance of the women dancers at the beginning of the video?'

output = runner({"video": video_path, "text_input": prompt}, length_penalty=1, \
        use_nucleus_sampling=True, num_beams=1, max_length=512, temperature=1.0)[0]
print(output)

/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
/home/jovyan/.mlspace/envs/video_vika/lib/python3.10/site-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be

At the beginning of the video, the six dancers are standing on stage wearing colorful costumes and holding their hands in their palms.


In [ ]:
import json
import os

import os
pwd = os.path.abspath(os.getcwd())

from tqdm import tqdm

with open("generic_qa.json") as file:
    gt_contents = json.load(file)

    # Create the output directory if it doesn't exist
if not os.path.exists('video_gpt_test'):
    os.makedirs('video_gpt_test')

output_list = []  # List to store the output results

video_formats = ['.mp4', '.avi', '.mov', '.mkv']

output_list = []    # Iterate over each sample in the ground truth file
for i, sample in tqdm(enumerate(gt_contents)):
    video_name = sample['video_name']
    sample_set = sample
    question = sample['Q']
    answer = sample['A']

    for fmt in video_formats:  # Added this line
        temp_path = os.path.join(pwd + "/videoChatgpt_test/", f"{video_name}{fmt}")
        if os.path.exists(temp_path):
            video_path = temp_path
            break

    try:
        # Run inference on the video and add the output to the list
        output = runner({"video": video_path, "text_input": question}, length_penalty=1, \
        use_nucleus_sampling=True, num_beams=1, max_length=512, temperature=1.0)[0]
    except:
        print ("yu")

    if (i% 100 == 0):
        print (question,'\n',  output, '\n', answer)
    sample_set['pred'] = output
    output_list.append(sample_set)

with open(os.path.join('video_gpt_test/', "generic_answer.json"), 'w') as file:
    json.dump(output_list, file)

1it [00:13, 13.01s/it]

What is the appearance of the women dancers at the beginning of the video? 
 At the beginning of the video, the group of women appear on stage performing belly dance in an orange and gold costume. 
 The women dancers are a group of five, wearing different colored belly-dancing outfits, holding their right hands up and facing away from the audience.


101it [22:09, 17.91s/it]

What happens to one of the women in the first team during the game? 
 During the game, one of the women on the first team has trouble keeping the shuttlecock and accidentally hits another player with it. This results in the game taking a temporary break until the injured player is able to retrieve the shuttlecock and resume the game. 
 One of the women in the first team walks out of the court and picks up a purse while walking towards the back of the court. She briefly speaks to one of her teammates while walking away.


201it [1:02:56, 11.26s/it]

What is happening on the tennis court at the beginning of the video? 
 The camera zooms in on a person's feet and shows a boy with a racket in his hands, preparing to serve. 
 The man with the racquet is standing on the right side of the court, throws the ball in the air, and hits it towards the other end of the court.


277it [1:31:00, 25.89s/it]

In [ ]:
import json
import os

import os
pwd = os.path.abspath(os.getcwd())

from tqdm import tqdm

with open("temporal_qa.json") as file:
    gt_contents = json.load(file)

    # Create the output directory if it doesn't exist
if not os.path.exists('video_gpt_test'):
    os.makedirs('video_gpt_test')

output_list = []  # List to store the output results

video_formats = ['.mp4', '.avi', '.mov', '.mkv']

output_list = []    # Iterate over each sample in the ground truth file
for i, sample in tqdm(enumerate(gt_contents)):
    video_name = sample['video_name']
    sample_set = sample
    question = sample['Q']
    answer = sample['A']

    for fmt in video_formats:  # Added this line
        temp_path = os.path.join(pwd + "/videoChatgpt_test/", f"{video_name}{fmt}")
        if os.path.exists(temp_path):
            video_path = temp_path
            break

    try:
        # Run inference on the video and add the output to the list
        output = runner({"video": video_path, "text_input": question}, length_penalty=1, \
        use_nucleus_sampling=True, num_beams=1, max_length=512, temperature=1.0)[0]
    except:
        print ("yu")

    if (i% 100 == 0):
        print (question,'\n',  output, '\n', answer)
    sample_set['pred'] = output
    output_list.append(sample_set)

with open(os.path.join('video_gpt_test/', "temporal_qa.json"), 'w') as file:
    json.dump(output_list, file)

1it [00:16, 16.37s/it]

What is the first activity that the boy wearing a red t-shirt and black shorts performs in the video? 
 The boy in the red t-shirt and black shorts jumps up on the uneven bars and begins swinging. 
 In the video, the boy wearing a red t-shirt and black shorts starts his routine by swinging on the bars and doing a handstand.


101it [24:01, 17.80s/it]

What is happening towards the end of the video? 
 Towards the end of the video, we see a girl bounce the ball to her other team member who then bounces the ball to the boy next to it, and then the girl bounces the ball to her friend who makes a successful layup shot. It appears that the game is ending. 
 Towards the end of the video, both teams stop playing and walk over to the middle of the basketball court.


201it [43:17,  8.77s/it]

...

382it [1:24:17, 15.10s/it]/home/jovyan/shares/SR004.nfs2/chekalina/LaVIT/VideoLaVIT/models/transform.py:200: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  start_indexs = np.where(np.array(frame_types)=='I')[0]
383it [1:24:18, 10.77s/it]

yu


401it [1:28:19, 14.28s/it]

What happens after the shuttlecock is missed by both players? 
 After the shuttlecock is missed by both players, the girl in the pink t-shirt and blue shorts serves the shuttlecock across the net. She misses out on the shuttlecock when her partner fails to return it. 
 After the shuttlecock is missed by both players, a child standing on the side picks it up and throws it at the girl. The camera then pans to people sitting on the side of chairs and one person is seen smiling. After this, the girls continue to play but keep missing the shuttlecock.


499it [1:51:19, 13.39s/it]


## Validation:

In [1]:
import json

import os
import gc
import re
import sys
import csv
import time
import tqdm
import json
import numpy as np
import pandas as pd
from copy import deepcopy
from pathlib import Path
from pprint import pprint
from typing import Optional, Any, List, Union, Tuple, Dict, Callable
from collections import defaultdict, OrderedDict, Counter

import openai
import asyncio
from tqdm.asyncio import tqdm_asyncio

with open(os.path.join('video_gpt_test/', "temporal_qa.json"), 'r') as file:
    output_list = json.load(file)

In [2]:
output_list[6]

{'Q': 'What are some of the different scenes shown in the video and how are they related temporally?',
 'A': 'The video begins with a man sailing a dinghy during daytime with cloudy skies and a visible pier in the distance. This is followed by clips of different dinghies sailing fast through the water. In one scene, a dinghy with red and white sails is shown, followed by a dinghy with a white sail, and then a few more dinghies sailing fast. Towards the end of the video, a man in a red wetsuit is shown maneuvering his dinghy, followed by clips of other sailors moving their dinghies around. The video ends with the man with the red sails leaning on his dinghy and maneuvering it, thus continuing the temporal sequence of events that began at the start of the video.',
 'video_name': 'v_dKJMP8EmZjw',
 'pred': 'The video features different scenes of sailboats and windsurfers on choppy waters. A red and white sailboat is seen turning over in the ocean at times while a man in a red jacket is see

In [3]:
len(output_list)

499

In [7]:
from openai import OpenAI, AsyncOpenAI

client = AsyncOpenAI(api_key='YOUR_API_KEY', base_url='http://0.0.0.0:23333/v1')

In [8]:
model_names = await client.models.list()
MODEL_NAME = model_names.data[0].id
MODEL_NAME

'unsloth/llama-3-8b-Instruct'

In [7]:
import ast

scores = []
for ind, elem in enumerate(output_list):
    question = elem['Q']
    answer = elem['A']
    pred = elem['pred']
    
    
    completion = await client.chat.completions.create(
                    model=MODEL_NAME,
                    messages=[
                    {
                        "role": "system",
                        "content":
                            "You are an intelligent chatbot designed for evaluating the temporal understanding of generative outputs for video-based question-answer pairs. "
                            "Your task is to compare the predicted answer with the correct answer and determine if they correctly reflect the temporal sequence of events in the video content. Here's how you can accomplish the task:"
                            "------"
                            "##INSTRUCTIONS: "
                            "- Focus on the temporal consistency between the predicted answer and the correct answer. The predicted answer should correctly reflect the sequence of events or details as they are presented in the video content.\n"
                            "- Consider synonyms or paraphrases as valid matches, but only if the temporal order is maintained.\n"
                            "- Evaluate the temporal accuracy of the prediction compared to the answer."
                    },
                    {
                        "role": "user",
                        "content":
                            "Please evaluate the following video-based question-answer pair:\n\n"
                            f"Question: {question}\n"
                            f"Correct Answer: {answer}\n"
                            f"Predicted Answer: {pred}\n\n"
                            "Provide your evaluation only as a temporal accuracy score where the temporal accuracy score is an integer value between 0 and 5, with 5 indicating the highest level of temporal consistency. "
                            "Please generate the response in the form of a Python dictionary string with keys 'score', where its value is the temporal accuracy score in INTEGER, not STRING."
                            "DO NOT PROVIDE ANY OTHER OUTPUT TEXT OR EXPLANATION. Only provide the Python dictionary string. "
                            "For example, your response should look like this: {''score': 4.8}."
                    }
                ]
    )

    # Convert response to a Python dictionary.
    if (ind%100 ==0):
        print (ind)
        print (print (question, answer, pred))
        print (ast.literal_eval(completion.choices[0].message.content)['score'])
    try:
        scores.append(ast.literal_eval(completion.choices[0].message.content)['score'])
    except:
        continue

0
What is the first activity that the boy wearing a red t-shirt and black shorts performs in the video? In the video, the boy wearing a red t-shirt and black shorts starts his routine by swinging on the bars and doing a handstand. The boy in the red t-shirt and black shorts jumps up on the uneven bars and begins swinging.
None
5
100
What is happening towards the end of the video? Towards the end of the video, both teams stop playing and walk over to the middle of the basketball court. Towards the end of the video, we see a girl bounce the ball to her other team member who then bounces the ball to the boy next to it, and then the girl bounces the ball to her friend who makes a successful layup shot. It appears that the game is ending.
None
5
200
What happens after the woman wearing an orange top walks up to the man in the kitchen? After the woman in the orange top walks up to the man washing dishes in the kitchen and throws a few items into the sink, the man steps away startled. In the 

In [8]:
import numpy as np
np.mean(scores)

2.4556451612903225

In [2]:
with open(os.path.join('video_gpt_test/', "generic_answer.json"), 'r') as file:
    output_list = json.load(file)

In [3]:
len(output_list)

1996

In [4]:
output_list[10]

{'Q': 'What type of drills are the children doing in the video?',
 'A': 'The children are doing different stances, punch and kick drills along with movement.',
 'video_name': 'v_jN-jnIgzd6k',
 'pred': 'The children are doing karate and practicing various moves and kicks. They seem to be following the instructions of an instructor who is standing in front of them.'}

## Correctness

In [9]:
import ast

scores = []
for ind, elem in enumerate(output_list):
    question = elem['Q']
    answer = elem['A']
    pred = elem['pred']
    
    
    completion = await client.chat.completions.create(
                    model=MODEL_NAME,
                    messages=[
                    {
                        "role": "system",
                        "content": 
                            "You are an intelligent chatbot designed for evaluating the factual accuracy of generative outputs for video-based question-answer pairs. "
                            "Your task is to compare the predicted answer with the correct answer and determine if they are factually consistent. Here's how you can accomplish the task:"
                            "------"
                            "##INSTRUCTIONS: "
                            "- Focus on the factual consistency between the predicted answer and the correct answer. The predicted answer should not contain any misinterpretations or misinformation.\n"
                            "- The predicted answer must be factually accurate and align with the video content.\n"
                            "- Consider synonyms or paraphrases as valid matches.\n"
                            "- Evaluate the factual accuracy of the prediction compared to the answer."
                    },
                    {
                        "role": "user",
                        "content":
                            "Please evaluate the following video-based question-answer pair:\n\n"
                            f"Question: {question}\n"
                            f"Correct Answer: {answer}\n"
                            f"Predicted Answer: {pred}\n\n"
                            "Provide your evaluation only as a factual accuracy score where the factual accuracy score is an integer value between 0 and 5, with 5 indicating the highest level of factual consistency. "
                            "Please generate the response in the form of a Python dictionary string with keys 'score', where its value is the factual accuracy score in INTEGER, not STRING."
                            "DO NOT PROVIDE ANY OTHER OUTPUT TEXT OR EXPLANATION. Only provide the Python dictionary string. "
                            "For example, your response should look like this: {''score': 4.8}."
                    }
                ]
    )

    # Convert response to a Python dictionary.
    if (ind%100 ==0):
        print (ind)
        print (print (question, answer, pred))
        print (ast.literal_eval(completion.choices[0].message.content)['score'])
    try:
        scores.append(ast.literal_eval(completion.choices[0].message.content)['score'])
    except:
        continue

0
What is the appearance of the women dancers at the beginning of the video? The women dancers are a group of five, wearing different colored belly-dancing outfits, holding their right hands up and facing away from the audience. At the beginning of the video, the group of women appear on stage performing belly dance in an orange and gold costume.
None
4
100
What happens to one of the women in the first team during the game? One of the women in the first team walks out of the court and picks up a purse while walking towards the back of the court. She briefly speaks to one of her teammates while walking away. During the game, one of the women on the first team has trouble keeping the shuttlecock and accidentally hits another player with it. This results in the game taking a temporary break until the injured player is able to retrieve the shuttlecock and resume the game.
None
0
200
What is happening on the tennis court at the beginning of the video? The man with the racquet is standing on

In [10]:
import numpy as np
np.mean(scores)

3.0105421686746987

## Context

In [ ]:
import ast

scores = []
for ind, elem in enumerate(output_list):
    question = elem['Q']
    answer = elem['A']
    pred = elem['pred']
    
    
    completion = await client.chat.completions.create(
                    model=MODEL_NAME,
                    messages=[
                    {
                        "role": "system",
                        "content":
                            "You are an intelligent chatbot designed for evaluating the contextual understanding of generative outputs for video-based question-answer pairs. "
                            "Your task is to compare the predicted answer with the correct answer and determine if the generated response aligns with the overall context of the video content. Here's how you can accomplish the task:"
                            "------"
                            "##INSTRUCTIONS: "
                            "- Evaluate whether the predicted answer aligns with the overall context of the video content. It should not provide information that is out of context or misaligned.\n"
                            "- The predicted answer must capture the main themes and sentiments of the video.\n"
                            "- Consider synonyms or paraphrases as valid matches.\n"
                            "- Provide your evaluation of the contextual understanding of the prediction compared to the answer."
                    },
                    {
                        "role": "user",
                        "content":
                            "Please evaluate the following video-based question-answer pair:\n\n"
                            f"Question: {question}\n"
                            f"Correct Answer: {answer}\n"
                            f"Predicted Answer: {pred}\n\n"
                            "Provide your evaluation only as a contextual understanding score where the contextual understanding score is an integer value between 0 and 5, with 5 indicating the highest level of contextual understanding. "
                            "Please generate the response in the form of a Python dictionary string with keys 'score', where its value is contextual understanding score in INTEGER, not STRING."
                            "DO NOT PROVIDE ANY OTHER OUTPUT TEXT OR EXPLANATION. Only provide the Python dictionary string. "
                            "For example, your response should look like this: {''score': 4.8}."
                    }
                ]
)

    # Convert response to a Python dictionary.
    if (ind%100 ==0):
        print (ind)
        print (print (question, answer, pred))
        print (ast.literal_eval(completion.choices[0].message.content)['score'])
    try:
        scores.append(ast.literal_eval(completion.choices[0].message.content)['score'])
    except:
        continue

0
What is the appearance of the women dancers at the beginning of the video? The women dancers are a group of five, wearing different colored belly-dancing outfits, holding their right hands up and facing away from the audience. At the beginning of the video, the group of women appear on stage performing belly dance in an orange and gold costume.
None
4
100
What happens to one of the women in the first team during the game? One of the women in the first team walks out of the court and picks up a purse while walking towards the back of the court. She briefly speaks to one of her teammates while walking away. During the game, one of the women on the first team has trouble keeping the shuttlecock and accidentally hits another player with it. This results in the game taking a temporary break until the injured player is able to retrieve the shuttlecock and resume the game.
None
0
200
What is happening on the tennis court at the beginning of the video? The man with the racquet is standing on

In [13]:
import numpy as np
np.mean(scores)

3.489073514602215

## Detailed orientation

In [14]:
import ast

scores = []
for ind, elem in enumerate(output_list):
    question = elem['Q']
    answer = elem['A']
    pred = elem['pred']
    
    
    completion = await client.chat.completions.create(
                    model=MODEL_NAME,
                    messages=[
                    {
                        "role": "system",
                        "content":
                            "You are an intelligent chatbot designed for evaluating the detail orientation of generative outputs for video-based question-answer pairs. "
                            "Your task is to compare the predicted answer with the correct answer and determine its level of detail, considering both completeness and specificity. Here's how you can accomplish the task:"
                            "------"
                            "##INSTRUCTIONS: "
                            "- Check if the predicted answer covers all major points from the video. The response should not leave out any key aspects.\n"
                            "- Evaluate whether the predicted answer includes specific details rather than just generic points. It should provide comprehensive information that is tied to specific elements of the video.\n"
                            "- Consider synonyms or paraphrases as valid matches.\n"
                            "- Provide a single evaluation score that reflects the level of detail orientation of the prediction, considering both completeness and specificity."
                    },
                    {
                        "role": "user",
                        "content":
                            "Please evaluate the following video-based question-answer pair:\n\n"
                            f"Question: {question}\n"
                            f"Correct Answer: {answer}\n"
                            f"Predicted Answer: {pred}\n\n"
                            "Provide your evaluation only as a detail orientation score where the detail orientation score is an integer value between 0 and 5, with 5 indicating the highest level of detail orientation. "
                            "Please generate the response in the form of a Python dictionary string with keys 'score', where its value is the detail orientation score in INTEGER, not STRING."
                            "DO NOT PROVIDE ANY OTHER OUTPUT TEXT OR EXPLANATION. Only provide the Python dictionary string. "
                            "For example, your response should look like this: {''score': 4.8}."
                    }
                ]
)

    # Convert response to a Python dictionary.
    if (ind%100 ==0):
        print (ind)
        print (print (question, answer, pred))
        print (ast.literal_eval(completion.choices[0].message.content)['score'])
    try:
        scores.append(ast.literal_eval(completion.choices[0].message.content)['score'])
    except:
        continue

0
What is the appearance of the women dancers at the beginning of the video? The women dancers are a group of five, wearing different colored belly-dancing outfits, holding their right hands up and facing away from the audience. At the beginning of the video, the group of women appear on stage performing belly dance in an orange and gold costume.
None
3
100
What happens to one of the women in the first team during the game? One of the women in the first team walks out of the court and picks up a purse while walking towards the back of the court. She briefly speaks to one of her teammates while walking away. During the game, one of the women on the first team has trouble keeping the shuttlecock and accidentally hits another player with it. This results in the game taking a temporary break until the injured player is able to retrieve the shuttlecock and resume the game.
None
2
200
What is happening on the tennis court at the beginning of the video? The man with the racquet is standing on

In [15]:
import numpy as np
np.mean(scores)

3.4090863453815263